In [15]:
import numpy as np
import matplotlib.pyplot as plt
import scipy
import os

from numpy import pi
from numpy import sin
from numpy import zeros
from numpy import r_
from scipy import signal
from scipy import misc # pip install Pillow
# import imageio
import matplotlib.pylab as pylab
from PIL import Image
import tarfile
import bz2
from zipfile import ZipFile

%matplotlib inline
pylab.rcParams['figure.figsize'] = (20.0, 7.0)

#### Helper Functions

In [16]:
#### Global Variables



rgb_to_YCbCr = np.array([[0.29900, 0.58700, 0.11400],
                       [-0.16874, -0.33126, 0.50000],
                       [0.50000, -0.41869, -0.08131]])

YCbCr_to_rgb = np.linalg.inv(rgb_to_YCbCr)


###common qunatizing matrices for dct 

Ql = np.array( [[16,12,14,14,18, 24, 49, 72],
                [11, 12, 13, 17, 22, 35, 64, 92],
                [10, 14, 16, 22, 37, 55, 78, 95],
                [16, 19, 24, 29, 56, 64, 87, 98],
                [24, 26, 40, 51, 68, 81, 103, 112],
                [40, 58, 57, 87, 109, 104, 121, 100],
                [51, 60, 69, 80, 103, 113, 120, 103],
                [61, 55, 56, 62, 77, 92, 101, 99]]
                ).T

Qc = np.array([[17, 18, 24, 47, 99, 99, 99, 99],
              [18, 21, 26, 66, 99, 99, 99, 99],
              [24, 26, 56, 99, 99, 99, 99, 99],
              [47, 66, 99, 99, 99, 99, 99, 99],
              [99, 99, 99, 99, 99, 99, 99, 99],
              [99, 99, 99, 99, 99, 99, 99, 99],
              [99, 99, 99, 99, 99, 99, 99, 99],
              [99, 99, 99, 99, 99, 99, 99, 99]]).T


In [42]:
#DCT Functions 
def dct2(a):
    return scipy.fftpack.dct( scipy.fftpack.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return scipy.fftpack.idct( scipy.fftpack.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')

#####################################################################################

##Padds an image so that its dimensions are divisible by 8.
def zero_pad(img):

    new_img = img
    
    if img.shape[0] % 8 != 0:
        zeros = np.zeros((8 - img.shape[0] % 8,img.shape[1],img.shape[2]))
        new_img = np.vstack((img,zeros))

    temp = new_img

    if img.shape[1] % 8 != 0:
        zeros = np.zeros((new_img.shape[0],8 - img.shape[1] % 8,new_img.shape[2]))
        temp = np.hstack((new_img,zeros))
    
    print("Zero Padded Shape: ", temp.shape)

    return temp

#####################################################################################

def helper_divide(A,Q):
    
    Y = np.zeros((8,8))
    for u, v in [(u, v) for u in range(8) for v in range(8)]:
        Y[u, v] = round(A[u, v] / Q[u, v])

    return Y 
def helper_mul(A,Q):
    
    Y = np.zeros((8,8))
    for u, v in [(u, v) for u in range(8) for v in range(8)]:
        Y[u, v] = A[u, v] * Q[u, v]

    return Y 

##Makes Sure that the values of the img are integers.
def quantize(U, q = 50.):

    s = U.shape
    Y = np.zeros(s)

    if q <= 50 and q > 0:

        alpha = 50.0 / q
    else:
        alpha = 2 - q / 50.0

    for i in r_[:U.shape[0]:8]:
        for j in r_[:U.shape[1]:8]:
            Y[i:(i+8),j:(j+8),0] = helper_divide(U[i:(i+8),j:(j+8),0], Ql)
#             Y[i:(i+8),j:(j+8),0] = (U[i:(i+8),j:(j+8),0] // (Ql))
    for k in range(1,3):
        for i in r_[:U.shape[0]:8]:
            for j in r_[:U.shape[1]:8]:
                Y[i:(i+8),j:(j+8),k] = helper_divide(U[i:(i+8),j:(j+8),k], alpha*Qc)
#                 Y[i:(i+8),j:(j+8),k] = (U[i:(i+8),j:(j+8),k] // (alpha*Qc))

    

    return Y

def iquantize(U, q=50.):
    
    s = U.shape
    Y = np.zeros(s)
    
    if q <= 50 and q>= 1:
        alpha = 50./q
    else:
        alpha = 2-q/50.0
    
    for i in r_[:U.shape[0]:8]:
        for j in r_[:U.shape[1]:8]:
#             Y[i:(i+8),j:(j+8),0] = (U[i:(i+8),j:(j+8),0] * (Ql))
            Y[i:(i+8),j:(j+8),0] = helper_mul(U[i:(i+8),j:(j+8),0], alpha*Ql)
    for k in range(1,3):
        for i in r_[:U.shape[0]:8]:
            for j in r_[:U.shape[1]:8]:
                Y[i:(i+8),j:(j+8),k] = helper_mul(U[i:(i+8),j:(j+8),k], alpha*Qc)
#                 Y[i:(i+8),j:(j+8),k] = (U[i:(i+8),j:(j+8),k] * (alpha* Qc))

    
    return Y


#####################################################################################


##converts from RGB To YCbCr 
def rgb2YCbCr(U):
    s = U.shape

    Y = np.zeros(s)
    for i in range(s[0]):
        for j in range(s[1]):

            Y[i,j] = np.dot(rgb_to_YCbCr,U[i,j]) + np.array([[0],[128],[128]])[:,0]
    return Y


##converts from YCbCr To RGB
def YCbCr2rgb(U):
    s = U.shape
    Y = np.zeros(s)
    for i in range(s[0]):
        for j in range(s[1]):
            Y[i,j] = np.dot(YCbCr_to_rgb,U[i,j] -  np.array([[0],[128],[128]])[:,0]) 
    return Y


#####################################################################################



## Zipping\unzipping the image with some extra info 

def zipping(img,min_val,max_val,beta,og_shape,img_name):

    ##creates a txt file that contains some extra info on the image such as dimensions. 
    output = open("rec_info.txt", "w")
    output.write(img_name + "|" + str(og_shape[0]) + "|" + str(og_shape[1]) + "|" + str(og_shape[2])+ "|" + str(min_val) + "|" + str(max_val) + "|" + str(beta))
    output.close()

    
    ##save Image 
    scipy.misc.imsave("rec.tiff",np.uint8(img))


    
    ###Encode Image 
    tar = tarfile.open("rec.tar.bz", "w:bz2")
    tar.add("rec.tiff")
    tar.add("rec_info.txt")
    
    tar.close()

    
    compressed_img_bytes = os.stat("rec.tar.bz").st_size
    if (compressed_img_bytes > 8000):
        print("Zipped File Too Big! " + str(compressed_img_bytes) + " Bytes")
    
    print("image zipped")

    return min_val, max_val


def unzipping():

    ## Unpack 
    tar = tarfile.open("rec_rec_toSend.tar.gz", "r")
    tar.extractall()
    tar.close()
    
    
    info = open("rec_info.txt", "r")
    info = info.read()
    info = info.split("|")
    print(info)
    
    img_name = info[0]
    og_shape = (int(info[1]),int(info[2]),int(info[3]))
    min_val = info[4]
    try:
        min_val = float(min_val)
    except:
        min_val = True
    
    max_val = float(info[5])
    beta = float(info[6])
    

    r = Image.open("rec.tiff")
    r = np.array(r)
    if min_val != True:
        r = ((r * max_val)/ 255) + min_val

    
    print("image unzipped")
    r = r.astype(int)
#     print(r)
    return r, img_name, og_shape, min_val, max_val, beta


### Preprocess Image

In [45]:
def compress_image(img, thresh, q,img_name,og_shape, modem_bw = 8000):
    
    #Downsample the Image
    img = img[::2, ::2]
    
    #Zero Padd To fit multiple of 8x8
    img = zero_pad(img)
    
    
    og_bytes = img.nbytes
    
    min_val = max_val = 0
    
    if og_bytes < modem_bw:
        print("returned original")
        min_val = True
        return img, min_val, max_val
     

    img = rgb2YCbCr(img)


    imsize = img.shape
    dct = np.zeros(imsize)

    
    # Do 8x8 DCT on image (in-place)
    for i in r_[:imsize[0]:8]:
        for j in r_[:imsize[1]:8]:
            dct[i:(i+8),j:(j+8)] = dct2( img[i:(i+8),j:(j+8)] )

        
            
    # Threshold
    if  og_bytes < modem_bw + 3000 :
        print("thresholding")
        dct_thresh = dct * (abs(dct) > (thresh*np.max(dct)))
    else: 
        dct_thresh = dct
    


    #Quantization
    if og_bytes > modem_bw and min_val != True:
        
        print("quantizing")
        dct_thresh = (quantize(dct_thresh,q))
        min_val = 0

        if np.min(dct_thresh) < 0:
            min_val = np.min(dct_thresh)
            dct_thresh += abs(np.min(dct_thresh))

        max_val = np.max(dct_thresh)

        dct_thresh = dct_thresh /  np.max(dct_thresh)
        dct_thresh *= 255       

    
    return dct_thresh, min_val,max_val

# Main

In [48]:
##Load Image
path = "./images/Isee.tiff"
img = misc.imread(path)

##
img_name = "./images/Isee.tiff"
original = img
beta = 50.
thresh = 0.0
og_shape = img.shape

print(og_shape)



##compress Image 
rec,min_val, max_val = compress_image(img, thresh, beta,img_name, og_shape)
zipping(rec,min_val,max_val,beta,og_shape,img_name)


print("Original Image Size: ", os.stat(img_name).st_size)
print("Compressed Image Size: ", os.stat("rec.tiff").st_size)
print("Zip File Size: ", os.stat("rec.tar.bz").st_size)


(200, 254, 3)
Zero Padded Shape:  (104, 128, 3)
quantizing
image zipped
Original Image Size:  179076
Compressed Image Size:  40076
Zip File Size:  2460


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:3: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel_launcher.py:136: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.
